In [0]:
%fs
ls "/mnt/olympics-data-2024"

path,name,size,modificationTime
dbfs:/mnt/olympics-data-2024/Athletes_Data.csv,Athletes_Data.csv,675175,1728727902000
dbfs:/mnt/olympics-data-2024/Coaches_dATA.csv,Coaches_dATA.csv,63031,1728727900000
dbfs:/mnt/olympics-data-2024/Medal_Data.csv,Medal_Data.csv,69357,1728727900000
dbfs:/mnt/olympics-data-2024/Team_Data.csv,Team_Data.csv,218715,1728727901000
dbfs:/mnt/olympics-data-2024/Total_Medals.csv,Total_Medals.csv,1750,1728727901000
dbfs:/mnt/olympics-data-2024/mfm_medal.csv,mfm_medal.csv,4095,1728734434000


In [0]:
Athletes=spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/mnt/olympics-data-2024/Athletes_Data.csv")
Coaches =spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/mnt/olympics-data-2024/Coaches_dATA.csv")
Medal=spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/mnt/olympics-data-2024/Medal_Data.csv")
Team=spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/mnt/olympics-data-2024/Team_Data.csv")
Total_Medals =spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/mnt/olympics-data-2024/Total_Medals.csv")
mfm_medal=spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/mnt/olympics-data-2024/mfm_medal.csv")
Athletes.show()

+-------+--------------------+------+--------+-----------------+-------------+
|   Code|                Name|Gender| Country|      Disciplines|Date_of_Birth|
+-------+--------------------+------+--------+-----------------+-------------+
|1532872|    ALEKSANYAN Artur|  Male| Armenia|        Wrestling|   1991-10-21|
|1532873|      AMOYAN Malkhas|  Male| Armenia|        Wrestling|   1999-01-22|
|1532874|     GALSTYAN Slavik|  Male| Armenia|        Wrestling|   1996-12-21|
|1532944|   HARUTYUNYAN Arsen|  Male| Armenia|        Wrestling|   1999-11-22|
|1532945|     TEVANYAN Vazgen|  Male| Armenia|        Wrestling|   1999-10-27|
|1532951|       ARENAS Lorena|Female|Colombia|        Athletics|   1993-09-17|
|1533112|     McKENZIE Ashley|  Male| Jamaica|             Judo|   1989-07-17|
|1533136|BASS BITTAYE Gina...|Female|  Gambia|        Athletics|   1995-05-03|
|1533176|     CAMARA Ebrahima|  Male|  Gambia|        Athletics|   1996-09-18|
|1533188| RUEDA SANTOS Lizeth|Female|  Mexico|      

In [0]:
Athletes.printSchema()

root
 |-- Code: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Disciplines: string (nullable = true)
 |-- Date_of_Birth: date (nullable = true)




### Filter and display all athletes representing the USA in the 2024 Olympics.

In [0]:
usa_athletes=Athletes.filter(Athletes.Country=="Armenia")
usa_athletes.show()

+-------+--------------------+------+-------+-------------------+-------------+
|   Code|                Name|Gender|Country|        Disciplines|Date_of_Birth|
+-------+--------------------+------+-------+-------------------+-------------+
|1532872|    ALEKSANYAN Artur|  Male|Armenia|          Wrestling|   1991-10-21|
|1532873|      AMOYAN Malkhas|  Male|Armenia|          Wrestling|   1999-01-22|
|1532874|     GALSTYAN Slavik|  Male|Armenia|          Wrestling|   1996-12-21|
|1532944|   HARUTYUNYAN Arsen|  Male|Armenia|          Wrestling|   1999-11-22|
|1532945|     TEVANYAN Vazgen|  Male|Armenia|          Wrestling|   1999-10-27|
|1541730|      CHALOYAN Davit|  Male|Armenia|             Boxing|   1997-09-30|
|1544413|    KARAPETYAN Garik|  Male|Armenia|      Weightlifting|   2003-06-11|
|1544417| KARAPETYAN Andranik|  Male|Armenia|      Weightlifting|   1995-12-15|
|1544424|    LALAYAN Varazdat|  Male|Armenia|      Weightlifting|   1999-05-01|
|1902794|   MKRTCHYAN Yervand|  Male|Arm

### Count the total number of medals won by male, female, and mixed teams for each country in the 2024 Olympics.

In [0]:
mfm_medal.show()

+-------------+----+------+-----+-----+
|      Country|Male|Female|Mixed|Total|
+-------------+----+------+-----+-----+
|United States|  52|    67|    7|  126|
|        China|  34|    50|    7|   91|
|    Australia|  20|    30|    3|   53|
|        Japan|  23|    18|    4|   45|
|  Netherlands|  11|    21|    2|   34|
|  New Zealand|   5|    14|    1|   20|
|        Italy|  23|    15|    2|   40|
|  South Korea|  10|    17|    5|   32|
|Great Britain|  30|    28|    7|   65|
|       Canada|   9|    17|    1|   27|
|France (host)|  38|    23|    3|   64|
|      Germany|  11|    15|    7|   33|
|       Brazil|   7|    12|    1|   20|
|        Kenya|   4|     7|    0|   11|
|      Hungary|  11|     8|    0|   19|
|      Ukraine|   6|     6|    0|   12|
|       Sweden|   5|     5|    1|   11|
|       Norway|   4|     4|    0|    8|
|      Algeria|   1|     2|    0|    3|
|     Slovenia|   1|     2|    0|    3|
+-------------+----+------+-----+-----+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import *

total_medal_won_by_gender = mfm_medal.select(
    sum("Male").alias("Male"),
    sum("Female").alias("Female"),
    sum("Mixed").alias("Mixed Medal Won"),
    sum("Total").alias("Total Medal Won"))
display(total_medal_won_by_gender)

Male,Female,Mixed Medal Won,Total Medal Won
499,479,65,1043


### Count the total number of medals Gold, Silver, and Bronze won by each country in the 2024 Olympics.

In [0]:
Total_Medals.show()

+-------------+----------+------------+------------+-----+
|      Country|Gold Medal|Silver Medal|Bronze Medal|Total|
+-------------+----------+------------+------------+-----+
|United States|        40|          44|          42|  126|
|        China|        40|          27|          24|   91|
|        Japan|        20|          12|          13|   45|
|    Australia|        18|          19|          16|   53|
|       France|        16|          26|          22|   64|
|  Netherlands|        15|           7|          12|   34|
|Great Britain|        14|          22|          29|   65|
|        Korea|        13|           9|          10|   32|
|        Italy|        12|          13|          15|   40|
|      Germany|        12|          13|           8|   33|
|  New Zealand|        10|           7|           3|   20|
|       Canada|         9|           7|          11|   27|
|   Uzbekistan|         8|           2|           3|   13|
|      Hungary|         6|           7|           6|   1

In [0]:
total_medals_by_medal_standings = Total_Medals.select(
    sum("Gold Medal").alias("Gold Medal"),
    sum("Silver Medal").alias("Silver Medal"),
    sum("Bronze Medal").alias("Bronze Medal"),
    sum("Total").alias("Total Medal Won")
)
display(total_medals_by_medal_standings)

Gold Medal,Silver Medal,Bronze Medal,Total Medal Won
329,330,384,1043


### Count the number of medals awarded in each sport during the 2024 Olympics.

In [0]:
joined_df = Total_Medals.join(Athletes, on="Country", how="inner")
medals_by_sport = joined_df.groupBy("Disciplines").agg(countDistinct("Total").alias("Total Medals Won"))
display(medals_by_sport)


Disciplines,Total Medals Won
Tennis,24
Boxing,27
Marathon Swimming,19
Golf,24
Rowing,25
Judo,27
Sailing,25
Swimming,27
Cycling BMX Freestyle,16
Basketball,13


### Average age of Athletic person at the 2024 Olympics 

In [0]:
from pyspark.sql.functions import *
current_age = Athletes.withColumn("age", year(current_date()) - year(col("Date_of_Birth")))
name_age_athletic_person = current_age.select("Name", "age")
display(name_age_athletic_person)
average_age = current_age.agg(avg("age").alias("average_age"))
display(average_age)


Name,age
ALEKSANYAN Artur,33
AMOYAN Malkhas,25
GALSTYAN Slavik,28
HARUTYUNYAN Arsen,25
TEVANYAN Vazgen,25
ARENAS Lorena,31
McKENZIE Ashley,35
BASS BITTAYE Gina Mariam,29
CAMARA Ebrahima,28
RUEDA SANTOS Lizeth,30


average_age
26.9678754611716


### Average age of Athletic person by Gender at the 2024 Olympics

In [0]:
from pyspark.sql.functions import *

MALE = Athletes.withColumn("age", year(current_date()) - year(col("Date_of_Birth"))).filter(col("Gender") == "Male")
average_age_Male = MALE.agg(avg("age").alias("average_age_male"))
average_age_Male.show()

+-----------------+
| average_age_male|
+-----------------+
|27.32237539766702|
+-----------------+



In [0]:
FEMALE = Athletes.withColumn("age", year(current_date()) - year(col("Date_of_Birth"))).filter(col("Gender") == "Female")
average_age_Female = FEMALE.agg(avg("age").alias("average_age_female"))
average_age_Female.show()

+------------------+
|average_age_female|
+------------------+
| 26.60018331805683|
+------------------+



### Compare the number of male and female athletes in 2024 Olympics

In [0]:
gender_count = Athletes.groupBy("gender").count()
gender_count.show()


+------+-----+
|gender|count|
+------+-----+
|Female| 5455|
|  Male| 5658|
+------+-----+



In [0]:
Team.show()

+--------------------+------+--------------+----------+------------+--------------------+------------+
|            TeamName|Gender|       Country|Disciplins|      Events|            Athletes|Num_Athletes|
+--------------------+------+--------------+----------+------------+--------------------+------------+
|People's Republic...|     M|         China|   Archery|  Men's Team|KAO Wenchao, LI Z...|           3|
|            Colombia|     M|      Colombia|   Archery|  Men's Team|ARCILA Santiago, ...|           3|
|              France|     M|        France|   Archery|  Men's Team|ADDIS Baptiste, C...|           3|
|       Great Britain|     M| Great Britain|   Archery|  Men's Team|HALL Conor, HALL ...|           3|
|               India|     M|         India|   Archery|  Men's Team|BOMMADEVARA Dhira...|           3|
|               Italy|     M|         Italy|   Archery|  Men's Team|MUSOLESI Federico...|           3|
|               Japan|     M|         Japan|   Archery|  Men's Team|FURUK

### Events count in 2024 Olympics 

In [0]:
most_events = Team.withColumn( "TeamName", col("TeamName")).groupBy("Events").count()
most_events.show()

+--------------------+-----+
|              Events|count|
+--------------------+-----+
|     Women's Madison|   15|
|       Dressage Team|   15|
|Women's 4 x 100m ...|   16|
|        Women's Four|   11|
|Women's Synchroni...|    8|
|   Men's Team Sprint|    8|
|Women's Synchroni...|    8|
|          Mixed Team|   46|
|Men's Canoe Doubl...|   15|
|Men's Synchronise...|    8|
|Men's 4 x 200m Fr...|   16|
|   Women's Foil Team|    8|
|     Mixed Multihull|   19|
|Lightweight Men's...|   18|
|    Men's Sabre Team|    8|
|        Women's Pair|   15|
|                NULL|  321|
|Marathon Race Wal...|   25|
|Lightweight Women...|   18|
|4 x 400m Relay Mixed|   16|
+--------------------+-----+
only showing top 20 rows



### distribution of athletes across different genders in various disciplines

In [0]:
from pyspark.sql.functions import col
df = Athletes.groupBy("Gender", "Disciplines").count().orderBy("Gender", "Disciplines")
display(df)


Gender,Disciplines,count
Female,3x3 Basketball,32
Female,"3x3 Basketball, Basketball",1
Female,Archery,64
Female,Artistic Gymnastics,94
Female,Artistic Swimming,106
Female,Athletics,982
Female,Badminton,87
Female,Basketball,143
Female,Beach Volleyball,48
Female,Boxing,124
